# State Farm
[State Farm Distracted Driver Detection | Kaggle](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data)

In [ ]:
%pwd

In [11]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

## Setup Bach Data

In [28]:
path = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/"

In [17]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

In [18]:
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 18424 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


## training

In [19]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_7 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 32, 222, 222)  896         batchnormalization_7[0][0]       
____________________________________________________________________________________________________
batchnormalization_8 (BatchNormal(None, 32, 222, 222)  64          convolution2d_4[0][0]            
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 32, 111, 111)  0           batchnormalization_8[0][0]       
___________________________________________________________________________________________

In [25]:
model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/10
18424/18424 [==============================] - 445s - loss: 2.6331 - acc: 0.2397 - val_loss: 1.7477 - val_acc: 0.4050
Epoch 2/10
18424/18424 [==============================] - 413s - loss: 2.3168 - acc: 0.3104 - val_loss: 0.9905 - val_acc: 0.7225
Epoch 3/10
18424/18424 [==============================] - 421s - loss: 2.0869 - acc: 0.3703 - val_loss: 0.8488 - val_acc: 0.7495
Epoch 4/10
18424/18424 [==============================] - 420s - loss: 1.8894 - acc: 0.4176 - val_loss: 0.6857 - val_acc: 0.8117
Epoch 5/10
18424/18424 [==============================] - 424s - loss: 1.7344 - acc: 0.4571 - val_loss: 0.5849 - val_acc: 0.8440
Epoch 6/10
18424/18424 [==============================] - 419s - loss: 1.6227 - acc: 0.4843 - val_loss: 0.5015 - val_acc: 0.8582
Epoch 7/10
18424/18424 [==============================] - 414s - loss: 1.5094 - acc: 0.5159 - val_loss: 0.4419 - val_acc: 0.8790
Epoch 8/10
18424/18424 [==============================] - 418s - loss: 1.4143 - acc: 0.5422 - val

## test data

In [30]:
test_batches = get_batches(path+'test', batch_size=2*batch_size, class_mode=None)
preds = model.predict_generator(test_batches, test_batches.nb_sample)

Found 79726 images belonging to 1 classes.


In [ ]:
#df = pd.read_csv(DATA_HOME_DIR+"submission2.csv")
#classes = sorted(batches.class_indices, key=batches.class_indices.get)
#preds = df[classes].values

In [31]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)
preds = do_clip(preds,0.93)

In [32]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.586313,0.018709,0.007778,0.007778,0.043606,0.007778,0.015010,0.007778,0.023829,0.296293
1,img_14887.jpg,0.278935,0.007778,0.013559,0.061090,0.084008,0.007778,0.007778,0.030671,0.007778,0.519838
2,img_62885.jpg,0.007778,0.007778,0.428875,0.019905,0.008673,0.011307,0.328163,0.007778,0.191944,0.007778
3,img_45125.jpg,0.007778,0.008297,0.008362,0.007778,0.007778,0.007778,0.007778,0.918671,0.010023,0.051053
4,img_22633.jpg,0.007778,0.007778,0.012447,0.007778,0.007778,0.007778,0.215847,0.027973,0.736934,0.007778


In [33]:
submission.to_csv("submission3.csv", index=False)

In [34]:
FileLink("submission3.csv")

/home/ubuntu/nbs/lesson4/submission3.csv